### Tuning the LSTM model ... 

File initially created 5/28/2025. 

I will start with a 14 day model.

I will need to tune by region. 

!pip install keras_tuner

In [1]:
##import cell makes its appearance once again...
%matplotlib inline
##so-called "math" related imports
#from netCDF4 import Dataset as ncread
import numpy as np
import pandas as pd
import math
from random import seed
from random import randint
from random import sample
from scipy.ndimage import gaussian_filter
from sklearn.metrics import brier_score_loss
import xarray as xr

import pickle

##plotting related imports
import matplotlib.pyplot as plt

#import tensorflow/keras related files
import tensorflow as tf    
#tf.compat.v1.disable_v2_behavior() # <-- HERE !

tf.device('/physical_device:GPU:0')

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Activation, Reshape, Flatten, LSTM, Dense, Dropout, Embedding, Bidirectional, GRU
from tensorflow.keras import Sequential
from tensorflow.keras import initializers, regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import constraints
from tensorflow.keras.layers import Layer, InputSpec

#tf.compat.v1.disable_eager_execution()
#import investigate

import keras_tuner as kt

2025-06-04 00:36:35.432419: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 00:36:35.463849: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-04 00:36:35.463870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-04 00:36:35.464808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 00:36:35.469888: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print(tf.__version__)

2.15.0


In [3]:
from EOF_def import EOF_def

In [4]:
##load in solvers for PC analysis to get the PCs for the model itself. 

In [5]:
# load input solvers
infile = open("../../eof_analysis/solvers/Usolver_14.p","rb",)
Usolver = pickle.load(infile)  ##pv on an isentropic surface, 350
infile.close()

infile = open("../../eof_analysis/solvers/EHFsolver_14.p","rb",)
EHFsolver = pickle.load(infile)  ##ZMehf vertical cross section along longitudes
infile.close()

infile = open("../../eof_analysis/solvers/GPHsolver_14.p","rb",)
GPHsolver = pickle.load(infile)  ##ZMehf vertical cross section along longitudes
infile.close()

In [6]:
## U
U_EOF_nw, U_EOF_nw2d, U_eigenv, U_VarEx, U_PC = EOF_def(Usolver, 10)

<class 'numpy.ndarray'> (8370, 10)


In [7]:
## EHF
EHF_EOF_nw, EHF_EOF_nw2d, EHF_eigenv, EHF_VarEx, EHF_PC = EOF_def(EHFsolver, 45)

<class 'numpy.ndarray'> (8370, 45)


In [8]:
## GPH
GPH_EOF_nw, GPH_EOF_nw2d, GPH_eigenv, GPH_VarEx, GPH_PC = EOF_def(GPHsolver, 10)

<class 'numpy.ndarray'> (8370, 10)


In [9]:
##remove PC 1
U_PC = U_PC[:, 1:]
EHF_PC = EHF_PC[:, 1:]
GPH_PC = GPH_PC[:, 1:]

In [10]:
# load output data
infile = open("../../eof_data/seus_anomtemps.p","rb",)
output = pickle.load(infile) 
infile.close()

In [11]:
##create one array of PCs
inputvar = np.concatenate((U_PC,EHF_PC,GPH_PC),axis=1) 

In [12]:
##make pandas dataframe for RF
input = pd.DataFrame(inputvar)
input

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,0.771931,-1.799955,1.313722,-1.630033,0.566290,1.589922,-0.172617,1.566368,0.659295,0.199667,...,-0.379306,1.641585,-0.743088,-0.070795,0.742423,0.454864,0.251855,0.795493,-0.417825,-1.029987
1,0.803999,-1.900528,1.245334,-1.692496,0.101666,2.042962,0.076518,1.506671,0.006843,0.238603,...,0.347120,1.752066,-0.985741,0.049346,0.764168,0.594407,0.432133,1.080915,-0.165571,-1.420320
2,0.872130,-2.032226,1.424598,-1.343837,0.161601,1.711995,0.503023,0.826614,-1.105172,0.270467,...,-0.585204,1.588777,-1.302961,0.097989,0.733291,0.651646,0.515277,0.912816,0.289325,-1.499576
3,0.867914,-2.054812,1.657540,-0.838510,0.746273,1.227320,0.679350,1.134812,-0.876215,0.232273,...,0.789144,1.273398,-1.416257,-0.048546,0.671334,0.746395,0.300846,0.709486,0.123002,-0.751547
4,0.787536,-2.011126,1.559197,-0.409157,1.239623,0.913107,0.641640,1.931937,-0.272880,0.127776,...,0.377190,1.043657,-1.369488,-0.219058,0.751703,1.057360,0.347496,0.404459,0.330464,0.215042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8365,0.415159,-0.170777,-0.531934,-0.397453,1.102362,-0.267771,0.062068,1.148350,-2.241040,0.180624,...,-0.535923,1.136943,-0.416174,0.562862,-0.354710,-0.763675,-0.224326,0.846225,-1.504478,-0.403197
8366,0.484640,-0.232514,-0.597377,0.039627,0.837373,-0.276658,0.205001,1.321532,-2.205615,0.232497,...,-0.670336,1.115493,-0.641672,0.649173,-0.423895,-0.589386,-0.034272,0.739777,-1.244535,-0.653119
8367,0.558206,-0.163820,-0.476075,0.480464,0.879917,-0.589342,-0.360315,1.203000,-2.164044,0.126000,...,-1.172690,0.839839,-1.031493,0.910849,-0.425116,-0.327954,0.842602,1.100344,-0.612106,-0.448852
8368,0.591930,0.343880,-0.538673,1.014618,0.200438,-0.391132,-0.570871,1.115725,-1.033758,0.161263,...,-0.749049,0.472475,-1.311037,0.590795,-0.424628,-0.503666,0.747163,1.328334,-0.101280,-0.280176


In [13]:
##need to change this based on lag 
## 14-days = [:, 47:]
## 20-days = [:, 53:]
## 30-days = [:, 63:]

temp = output.reshape(62, 182)
temp = temp[:, 43:]
temp.shape

(62, 139)

In [14]:
#selected indices
##Europe
#Index([1, 56, 0, 54, 9, 57, 55, 3, 26, 4], dtype='int64')

##Nova
#Index([1, 56, 0, 55, 2, 53, 54, 9, 4, 22], dtype='int64')

##South
#Index([1, 56, 54, 53, 55, 4, 3, 21, 14, 0], dtype='int64')

In [15]:
#create pd datafram of selected feature columns.

sele_ind_df = input[[1, 56, 54, 53, 55, 4, 3, 21, 14, 0]]

In [16]:
#look at new input array
sele_ind_df

,1,56,54,53,55,4,3,21,14,0
0,-1.799955,0.742423,-0.743088,1.641585,-0.070795,0.566290,-1.630033,0.471726,0.035413,0.771931
1,-1.900528,0.764168,-0.985741,1.752066,0.049346,0.101666,-1.692496,0.557982,0.016162,0.803999
2,-2.032226,0.733291,-1.302961,1.588777,0.097989,0.161601,-1.343837,0.245462,-0.013173,0.872130
3,-2.054812,0.671334,-1.416257,1.273398,-0.048546,0.746273,-0.838510,0.294595,-0.024929,0.867914
4,-2.011126,0.751703,-1.369488,1.043657,-0.219058,1.239623,-0.409157,-0.185676,-0.183715,0.787536
...,...,...,...,...,...,...,...,...,...,...
8365,-0.170777,-0.354710,-0.416174,1.136943,0.562862,1.102362,-0.397453,1.902774,0.188527,0.415159
8366,-0.232514,-0.423895,-0.641672,1.115493,0.649173,0.837373,0.039627,1.512352,0.213924,0.484640
8367,-0.163820,-0.425116,-1.031493,0.839839,0.910849,0.879917,0.480464,0.629668,-0.062387,0.558206
8368,0.343880,-0.424628,-1.311037,0.472475,0.590795,0.200438,1.014618,0.039851,-0.039619,0.591930


In [17]:
#turn input array into (# samples, 14 days, 10 features)
sele_ind_data = sele_ind_df.values
sele_ind_data1 = sele_ind_data.reshape(62,135,10)

temp_flat = temp.flatten()

In [18]:
sele_ind_data.shape

(8370, 10)

In [19]:
#create timeseries data arrays for PCs 
new_input =  np.empty((62,126,10,10))
new_output = np.empty((62,126))

for i in range(0,62):
    for j in range(0,126):
        new_input[i,j,:,:] = sele_ind_data1[i,j:j+10,:]
        new_output[i,j] = temp[i,j]

In [20]:
new_input = np.array(new_input)
new_output = np.array(new_output)
print(new_input.shape)
print(new_output.shape)

(62, 126, 10, 10)
(62, 126)


In [21]:
frac_ind = round((62*126)*0.175)
frac_end = round((62*126)-frac_ind)
frac_end

6445

In [22]:
frac_ind

1367

In [23]:
##Set X_all and Y_all datasets
X_all = np.copy(new_input.reshape((62*126),10,10))
Y_all = np.copy(new_output.flatten())


##training data partition out
X_tri = X_all[:frac_end,:]
Y_tri = Y_all[:frac_end]

#testing data partition out
X_tes = X_all[frac_end:,:]
Y_tes = Y_all[frac_end:]

#Convert the Y array into a categorical array. This means we will create one-hot vector labels for all of the inputs.
# The one-hot vectors have an index for each possible output category (two in our case)
# A "1" is put in the index corresponding to the category to which the sample belongs
Y_all = keras.utils.to_categorical(Y_all)
Y_tri = keras.utils.to_categorical(Y_tri)
Y_tes= keras.utils.to_categorical(Y_tes)

In [24]:
Y_all.shape

(7812, 2)

In [25]:
##checking my data for NaN of Infs because I need to make sure this doesn't cause
#the model to throw back no loss

if np.any(np.isnan(X_all)) or np.any(np.isinf(X_all)):
    print("NaN or Inf values found in X_all!")

if np.any(np.isnan(Y_all)) or np.any(np.isinf(Y_all)):
    print("NaN or Inf values found in Y_all!")

### Begin establishing specifics of model.

In [26]:
##checking to make sure shape was properly one hot encoded
Y_all.shape

(7812, 2)

In [27]:
##class weight creator for the instance where now I have a four dimensional output array 
def class_weight_creator(Y):
    class_dict = {}
    Y_reshaped = Y.reshape(-1, Y.shape[-1])
    weights = np.max(np.sum(Y_reshaped, axis=0)) / np.sum(Y_reshaped, axis=0)
    for i in range(Y.shape[-1] ):
        class_dict[i] = weights[i]
        
    return class_dict

In [28]:
##number of input nodes
numb_int = X_all.shape[1:]

'''##fraction of training data
X_validation = X_tri[0:frac_ind]
Y_validation = Y_tri[0:frac_ind]
        
X_train = X_tri[frac_ind:len(X_tri)]
Y_train = Y_tri[frac_ind:len(Y_tri)]

X_test = X_tes
Y_test = Y_tes'''

'##fraction of training data\nX_validation = X_tri[0:frac_ind]\nY_validation = Y_tri[0:frac_ind]\n        \nX_train = X_tri[frac_ind:len(X_tri)]\nY_train = Y_tri[frac_ind:len(Y_tri)]\n\nX_test = X_tes\nY_test = Y_tes'

In [29]:
#Y_train.shape

In [30]:
numb_int

(10, 10)

### Model Architecture. 

In [31]:
######
batch_size = 64 #The number of samples the network sees before it backpropagates (batch size)
epochs = 50 #The number of times the network will loop through the entire dataset (epochs)
shuffle = True #Set whether to shuffle the training data so the model doesn't see it sequentially 
verbose = 2 #Set whether the model will output information when trained (0 = no output; 2 = output accuracy every epoch)

In [32]:
def categorical_focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        # Clip predictions to prevent log(0)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        # Cross-entropy loss
        cross_entropy = -y_true * tf.math.log(y_pred)

        # Compute focal loss scaling factor
        focal_factor = tf.pow(1 - y_pred, gamma)
        loss = alpha * focal_factor * cross_entropy

        return tf.reduce_sum(loss, axis=1)  # sum across classes per sample

    return loss

In [33]:
def tune_LSTM(hp): 
    ##set ranges for hyperparameters
    n1 = hp.Int('n1', min_value=32, max_value=96, step=32)
    n2 = hp.Int('n2', min_value=4, max_value=32, step=4)
    n3 = hp.Int('n3', min_value=16, max_value=32, step=8)
    
    reg1 = hp.Float('reg1', min_value=0.001, max_value=0.9, step=0.001)
    #reg2 = hp.Float('reg2', min_value=0.01, max_value=0.9, step=0.01)
    reg3 = hp.Float('reg3', min_value=0.001, max_value=0.9, step=0.001)
    
    #alpha = hp.Float('alpha', 0.4, 0.8, step=0.1)
    #gamma = hp.Float('gamma', 0.2, 1.0, step=0.2)
    
    
    
    input_tensor = Input(shape=(10, 10))
    layer1 = layers.RNN(
        layers.LSTMCell(n1, activation='tanh', use_bias=True,
                          kernel_initializer='glorot_uniform',
                          kernel_regularizer=keras.regularizers.l2(l2= reg1)),
        return_sequences=True)(input_tensor)
    layer2 = layers.RNN(
        layers.LSTMCell(n2, activation='tanh', use_bias=True,
                          kernel_initializer='glorot_uniform',
                          kernel_regularizer=keras.regularizers.l2(l2= reg1)))(layer1)
    layer3 = layers.Dense(n3, activation='relu',use_bias=True,
                          kernel_initializer='he_normal',bias_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(l2=reg3))(layer2)

    output_tensor = layers.Dense(2, activation='softmax',)(layer3)

    model = Model(input_tensor, output_tensor)
    
    lr = hp.Float('learning_rate', min_value=0.0001, max_value=0.1)
    
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
    
    #loss = categorical_focal_loss(alpha=alpha, gamma=gamma)
    #decay_rate = lr / epochs
    #momentum = 0.9

    model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=[keras.metrics.categorical_accuracy],)
                            
    return model

In [34]:
##the LSTM layers will have EXACTLY the same regularizer

In [35]:
tf.keras.backend.clear_session()

In [36]:
###################################################

In [37]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True,verbose=1)

In [38]:
#rather than CVs, I am going to introduce k-fold validation instead. 

In [39]:
from sklearn.model_selection import KFold

def run_kfold_tuning(X, Y, n_splits, epochs, batch_size):
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    all_best_hps = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        fold_id = str(fold+1)
        
        X_train, X_val = X[train_idx], X[val_idx]
        Y_train, Y_val = Y[train_idx], Y[val_idx]
        
        tuner = kt.BayesianOptimization(
            tune_LSTM,
            objective=[kt.Objective('val_loss', 'min'), kt.Objective('val_categorical_accuracy', 'max')],
            max_trials=150,
            directory='tuning',
            project_name=str("Eur_CV_Tk5_"+fold_id))
        
        tuner.search(
            X_train, Y_train,
            epochs=epochs,
            validation_data=(X_val, Y_val),
            batch_size=batch_size,callbacks=[stop_early],
            verbose=1
        )
        
        best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
        all_best_hps.append(best_hp)
        
    return all_best_hps

In [40]:
#run tuning
best_hps = run_kfold_tuning(X_tri, Y_tri, 5, 30, batch_size)

Trial 4 Complete [00h 00m 15s]
multi_objective: -0.004714846611022949

Best multi_objective So Far: -0.004714846611022949
Total elapsed time: 00h 01m 05s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
64                |32                |n1
12                |16                |n2
32                |24                |n3
0.414             |0.196             |reg1
0.207             |0.254             |reg3
0.005762          |0.0028757         |learning_rate

Epoch 1/30
81/81 [==============================] - 2s 18ms/step - loss: 9.8665 - categorical_accuracy: 0.6633 - val_loss: 2.1764 - val_categorical_accuracy: 0.6501
Epoch 2/30
81/81 [==============================] - 1s 14ms/step - loss: 1.2608 - categorical_accuracy: 0.6693 - val_loss: 0.8104 - val_categorical_accuracy: 0.6501
Epoch 3/30
81/81 [==============================] - 1s 14ms/step - loss: 0.6953 - categorical_accuracy: 0.6693 - val_loss: 0.6619 - val_categorical_accuracy: 0.6501
Epoch 4/3

KeyboardInterrupt: 

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
lstm_model = tuner.hypermodel.build(best_hps)
history = lstm_model.fit(X_train_cv,Y_train_cv, validation_data=(X_val_cv, Y_val_cv),
                    batch_size=batch_size, epochs=epochs, shuffle=shuffle,class_weight = class_weight)


In [ ]:
train_loss=lstm_model.history.history['loss']
val_loss=lstm_model.history.history['val_loss']
    
cat_acc=lstm_model.history.history['categorical_accuracy']
val_acc=lstm_model.history.history['val_categorical_accuracy']
        
pred = lstm_model.predict(X_all)
pred_val = lstm_model.predict(X_val_cv)
pred_train = lstm_model.predict(X_train_cv)
pred_test = lstm_model.predict(X_tes)

# Look at the optimization history
fig, (ax1,ax2) = plt.subplots(1,2, figsize=plt.figaspect(0.25))
ax1.plot(train_loss, label='Training loss')
ax1.plot(val_loss, label='Validation loss')
ax1.set_yscale('log')
ax1.set_title('loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('loss')
ax1.legend()

ax2.plot(cat_acc, label='Training ACC')
ax2.plot(val_acc, label='Validation ACC')
ax2.set_title('Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Acc')
ax2.legend